# Building A Portfolio

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')
import sys

In [ ]:
sys.setrecursionlimit(2000)  # Increase the recursion limit

In [ ]:
#get potfolio weights
portfolio_data= pd.read_csv('/content/portfolio_allocation.csv')

In [ ]:
weights = portfolio_data[['Ticker', 'Weight']].sort_values(by='Ticker', ascending=True)

In [ ]:
portfolio_data['Weight'].sum() #veify that the weight = 1

0.9999999999999962

In [ ]:
weights['Weight'] = weights['Weight'] * 100

In [ ]:
#Getting 10 stocks for the Portfolio
start_date = '2023-12-29'
end_date = '2024-09-21'

stocks = ['NVDA', 'MSFT', 'JPST', 'IWB', 'GOOGL', 'ABNB', 'ACGL', 'ADT', 'AGO',
 'AMAT', 'AMD', 'BLDR', 'BLK', 'BROS', 'BX', 'CALM', 'CAT', 'CELH', 'COIN',
 'COST', 'CSGP', 'CSIQ', 'CVX', 'DECK', 'DIS', 'DVA', 'EA', 'EW', 'F', 'FNV',
 'FTRE', 'GDDY', 'GM', 'GS', 'IIPR', 'ISRG', 'JPM', 'KKR', 'KLAC', 'LBRT',
 'LH', 'LLY', 'LRN', 'MA', 'MANH', 'MCD', 'MCO', 'MEDP', 'META', 'MLI', 'MLM',
 'MSCI', 'MU', 'NFLX', 'NTCT', 'NTDOY', 'OPXS', 'PG', 'PM', 'PSTG', 'QCOM',
 'RCL', 'REGN', 'RWT', 'SBUX', 'SEDG', 'SONY', 'SPGI', 'SPOT', 'TMUS', 'TXN',
 'UNH', 'UNP', 'URI', 'V', 'VEEV', 'XEL', 'XYL']
 #Portfolio

#HNR was aquired
# And ALAB destroys the symetry of the porfolio, they IPO this year, and I have data for little over a year now

markets = ['^GSPC', 'DAX']  #Market

stocks_df = yf.download(stocks, start=start_date, end=end_date)['Adj Close'] #list of Adj Closing

markets_df = yf.download(markets, start=start_date, end=end_date)['Adj Close']

Rfr = yf.download('^TNX', start=start_date, end=end_date)['Adj Close'] #Treasury Bill

[*********************100%***********************]  78 of 78 completed
[*********************100%***********************]  2 of 2 completed
[*********************100%***********************]  1 of 1 completed


In [ ]:
mr = (markets_df['^GSPC'].iloc[-1] - markets_df['^GSPC'].iloc[0]) / markets_df['^GSPC'].iloc[0] * 100

# Calculating Stock Returns

* Calculating 5 year return of the stock and the markets


In [ ]:
returns = stocks_df.pct_change(1) #Calculate returns

In [ ]:
returns = returns.dropna() #drop missing values

In [ ]:
# Calculating daily returns for the SPY
markets_returns = markets_df.pct_change(1)

In [ ]:
markets_returns = markets_returns.dropna()

In [ ]:
markets_returns

Ticker,DAX,^GSPC
Date,,
2024-01-02 00:00:00+00:00,-0.010049,-0.005661
2024-01-03 00:00:00+00:00,-0.015062,-0.008016
2024-01-04 00:00:00+00:00,0.005652,-0.003428
2024-01-05 00:00:00+00:00,0.001983,0.001826
2024-01-08 00:00:00+00:00,0.012867,0.014115
...,...,...
2024-09-16 00:00:00+00:00,0.006478,0.001257
2024-09-17 00:00:00+00:00,-0.004960,0.000265
2024-09-18 00:00:00+00:00,0.004748,-0.002896


In [ ]:
spy_avg_return = markets_df['^GSPC'].pct_change() #calculating the average return for the SPY

spy_avg_return = spy_avg_return.mean()

# Expected Return

* Expected returns using CAPM
* Arbitrage Pricing Theory AP Models
* Arbitrage Pricing Theory APT using model suggest multiple factors affect returns

### CAPM Model
- Works on the premise that the return on any asset us based exclusively on the asset's relationship with the market

- Formula: Expected Return = rf+ Bj (E[rm] - rf)
- Where:
- E[rm] = Expected return on the stock market
- rf = Risk Free rate (e.g Yield of T-bills or T-Bonds)
- Bj = Systematic Risk (Market Risk) of stock j

In [ ]:
stocks_ß = []
tickers = []

market_variance = np.var(markets_returns['^GSPC'])

for stock in returns.columns:
    covariance = np.cov(returns[stock], markets_returns['^GSPC'])[0,1] #[0,1] extract the covariance between the stock and the market, this then divided by the variance of the market

    beta = covariance / market_variance

    tickers.append(stock)
    stocks_ß.append(beta)

In [ ]:
portfolio_df = pd.DataFrame({'Ticker': tickers, 'Beta': stocks_ß })

In [ ]:
portfolio_df

,Ticker,Beta
0,ABNB,1.428892
1,ACGL,0.112902
2,ADT,0.994816
3,AGO,0.419417
4,AMAT,2.423767
...,...,...
73,URI,1.674282
74,V,0.569786
75,VEEV,0.816459
76,XEL,-0.059293


In [ ]:
betas = portfolio_df['Beta']

In [ ]:
def CAPM(Rfr, mr, betas):

    #Annual Market Return daily * 250
    mr =  (markets_df['^GSPC'].iloc[-1] - markets_df['^GSPC'].iloc[0]) / markets_df['^GSPC'].iloc[0] * 100

    #Risk-free rate
    Rfr = Rfr[-1] / 100

    # Beta
    betas = portfolio_df['Beta']

    return Rfr + betas * (mr - Rfr)

portfolio_df['CAPM'] = CAPM(Rfr, mr, portfolio_df['Beta'])

In [ ]:
portfolio_df

,Ticker,Beta,CAPM
0,ABNB,1.428892,27.925378
1,ACGL,0.112902,2.240821
2,ADT,0.994816,19.453387
3,AGO,0.419417,8.223157
4,AMAT,2.423767,47.342641
...,...,...,...
73,URI,1.674282,32.714718
74,V,0.569786,11.157952
75,VEEV,0.816459,15.972349
76,XEL,-0.059293,-1.119961


In [ ]:
avg_returns = returns.mean() * 100

In [ ]:
avg_returns.name = 'Avg Return'

In [ ]:
portfolio_df = pd.merge(portfolio_df, avg_returns, left_on='Ticker', right_index=True, how='left')

## Measuring Risk

There are a nuumber of risk measurments: the stdandard deviation or variance of portfolio return, the semivarinace of returns, the tracking error of returns, the VaR (Value at Risk) of the returns the correlation and covarinace of the portfolio and the beta of the portfolio.

# Standard Deviation

The Standard Deviation measures how much the returns of a portfolio move around the average return. The standard deviation grows a returns move furthere above or further below the average

In [ ]:
#Standard Deviation
std_dev = np.std(returns)

In [ ]:
std_dev #std grows as returns move further above or further below the mean

,0
Ticker,
ABNB,0.021323
ACGL,0.012392
ADT,0.023119
AGO,0.015803
AMAT,0.027050
...,...
URI,0.023439
V,0.009680
VEEV,0.018258


In [ ]:
#std_dev as a data frame
std_dev = pd.DataFrame(std_dev)

In [ ]:
portfolio_df = pd.merge(portfolio_df, std_dev, left_on='Ticker', right_index=True, how='left')

In [ ]:
portfolio_df.rename(columns={0: 'Std Dev'}, inplace=True)

In [ ]:
portfolio_df['Std Dev'] = portfolio_df['Std Dev'].round(4) * 100

# Semi Standard Deviation


In [ ]:
avg_returns = returns.mean() #average daily return

In [ ]:
treshold = avg_returns
below_treshold = returns[returns <= treshold]

In [ ]:
semi_std = below_treshold.std(ddof=1)

semi_std = pd.DataFrame(semi_std) #covert into Data Frame

semi_std.columns = ['Semi Std Dev'] #name the columns

semi_std.reset_index(inplace=True) #reset the index

semi_std = semi_std['Semi Std Dev'].round(4) * 100 #  Calculate the STD

In [ ]:
semi_std.drop(columns=['Ticker'], inplace=True)

In [ ]:
semi_std

,Semi Std Dev
0,1.74
1,0.76
2,1.92
3,0.90
4,1.92
...,...
73,1.45
74,0.75
75,1.32
76,1.27


In [ ]:
# for x in semi_std:
    # semi_std[x] = semi_std[x].fillna(0)

In [ ]:
portfolio_df = pd.concat([portfolio_df, semi_std], axis=1)

In [ ]:
for x in portfolio_df[['Beta','CAPM',	'Avg Return', 'Std Dev',	'Semi Std Dev']]:
    portfolio_df[x] = portfolio_df[x].round(3)

# Tracking Error


In [ ]:
portfolio_df

,Ticker,Beta,CAPM,Avg Return,Std Dev,Semi Std Dev
0,ABNB,1.429,27.925,0.003,2.13,1.74
1,ACGL,0.113,2.241,0.240,1.24,0.76
2,ADT,0.995,19.453,0.074,2.31,1.92
3,AGO,0.419,8.223,0.063,1.58,0.90
4,AMAT,2.424,47.343,0.133,2.71,1.92
...,...,...,...,...,...,...
73,URI,1.674,32.715,0.203,2.34,1.45
74,V,0.570,11.158,0.057,0.97,0.75
75,VEEV,0.816,15.972,0.077,1.83,1.32
76,XEL,-0.059,-1.120,0.046,1.45,1.27


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#add weight into portfolio_df
portfolio_df = pd.merge(portfolio_df, weights, on='Ticker', how='inner')

In [ ]:
portfolio_df

,Ticker,Beta,CAPM,Avg Return,Std Dev,Semi Std Dev,Weight
0,ABNB,1.429,27.925,0.003,2.13,1.74,1.622475
1,ACGL,0.113,2.241,0.240,1.24,0.76,1.723909
2,ADT,0.995,19.453,0.074,2.31,1.92,0.694525
3,AGO,0.419,8.223,0.063,1.58,0.90,1.247948
4,AMAT,2.424,47.343,0.133,2.71,1.92,1.088174
...,...,...,...,...,...,...,...
73,URI,1.674,32.715,0.203,2.34,1.45,1.460086
74,V,0.570,11.158,0.057,0.97,0.75,1.225559
75,VEEV,0.816,15.972,0.077,1.83,1.32,0.555518
76,XEL,-0.059,-1.120,0.046,1.45,1.27,0.336520


# STOP HERE

In [ ]:
d = data[['AMAT', 'AMGN', 'AMZN', 'CCJ', 'LMT', 'LULU', 'PANW', 'XOM']]

NameError: name 'data' is not defined

In [ ]:
stats = d.calc_stats()
stats.display()

In [ ]:
SOFI_stats = SOFI.calc_stats()
SOFI_stats.display()

In [ ]:
PLTR_stats = PLTR.calc_stats()
PLTR_stats.display()

In [ ]:
weights = np.array([0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10, 0.10])

In [ ]:
#mean daily returns and covariance of daily returns
mean_daily_returns = returns.mean()
cov_matrix = returns.cov()

In [ ]:
pf_return = np.sum(mean_daily_returns * weights) * 252
pr_return = round(pf_return, 3)

In [ ]:
pf_std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
pf_std_dev = round(pf_std_dev,3)

In [ ]:
print('Expected annualized return: ' + '{:.1%}'.format(pf_return))
print("Volatility: " + "{:.1%}".format(pf_std_dev))

In [ ]:
#calculate expected returns and sample covariance
expected_returns = expected_returns.mean_historical_return(data)

In [ ]:
expected_returns

In [ ]:
risk_model = risk_models.sample_cov(data)

In [ ]:
risk_model

In [ ]:
#Optimize for maximal sharp ratio
ef = EfficientFrontier(expected_returns, risk_model)
raw_weight = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)
perf = ef.portfolio_performance(verbose=True)

In [ ]:
ef = EfficientFrontier(expected_returns, risk_model,  weight_bounds=(-1,1))
pf = ef.efficient_return(target_return=perf[0])
print(pf)
print()
perf = ef.portfolio_performance(verbose=True)

In [ ]:
current_price = data.iloc[-1] #last price of the stock

In [ ]:
#cretate dictionary for the stock weights
w_data = {'AMZN': 0.10, 'CCJ': 0.10, 'LMT': 0.10, 'PLTR': 0.10, 'PANW': 0.10, 'LULU': 0.10, 'SOFI': 0.10, 'XOM': 0.10, 'AMGN': 0.10, 'AMAT': 0.10}

Using the Efficient Frontier for portfolio optimization can be limiting, as it tends to select the five stocks with the highest returns, making the portfolio less risk-averse

In [ ]:
da = DiscreteAllocation(w_data, current_price, total_portfolio_value=10000)
allocation, leftover = da.greedy_portfolio()
print('Discrete allocation:', allocation)
print('Funds remaining: ${:.2f}'.format(leftover))